In [ ]:
# Importation des bibliothèques 
import geopandas as gpd
import rasterstats
import pandas as pd
from rasterstats import zonal_stats
import os 
from osgeo import gdal
import numpy as np
from collections import defaultdict
from my_function import apply_decision_rules, compute_confusion_matrix_with_plots

In [ ]:
# Définition des cheminss d'accès 
my_folder = '/home/onyxia/work/Depot_Git/results/data'
sample_filename = os.path.join(my_folder, 'sample/Sample_BD_foret_T31TCJ.shp')
image_filename = os.path.join (my_folder,'classif/carte_essences_echelle_pixel.tif')

In [ ]:
# Utilisation de zonal_stats pour obtenir le total des pixels par polygone
zonal_statistics = zonal_stats(
    sample_filename,
    image_filename,
    stats=["count"],  # Nombre total de pixels par polygone
    categorical=True  # Activer le mode catégoriel pour extraire les classes
)

# Chargement de raster pour extraire les classes
raster = gdal.Open(image_filename)
band = raster.GetRasterBand(1)  
raster_data = band.ReadAsArray()

# Boucle pour extraire les classes par polygone
# liste de dictionnaires pour stocker les pourcentages des classes par polygone
polygon_classes_percentages = []

# Parcourir les statistiques zonales
for idx, stats in enumerate(zonal_statistics):
    polygon_id = idx + 1  
    total_pixels = stats["count"]  # Nombre total de pixels dans le polygone

    # Initialisation d'un dictionnaire pour stocker les pourcentages des classes pour ce polygone
    class_percentages = {}
    
    # Parcourir chaque classe dans le polygone
    for class_value, pixel_count in stats.items():
        if class_value == "count":  # Ignorer le total
            continue
        
        # Calcul du pourcentage
        percentage = (pixel_count / total_pixels) * 100
        class_percentages[class_value] = percentage

    #  Ajout de résultats pour ce polygone
    polygon_classes_percentages.append({
        "polygon_id": polygon_id,
        "class_percentages": class_percentages
    })

# Affichage des résultats
for polygon_result in polygon_classes_percentages:
    print(f"Polygone {polygon_result['polygon_id']} :")
    for class_value, percentage in polygon_result["class_percentages"].items():
        print(f"  Classe {class_value}: {percentage:.2f}%")
        
# Transformation de resulats sous forme d'un dataframe pour qu'elle soit utilisée par la suite dans la fonction d'arbre de décision 
df_polygon_classes_percentages= pd.DataFrame(polygon_classes_percentages)
df_polygon_classes_percentages.head(5)

In [ ]:
import pandas as pd
import geopandas as gpd

def calculate_surface(samples_path, index):
    """
    Calcular la superficie de un polígono a partir de las muestras.

    Argumentos:
    - samples_path: Ruta al archivo de muestras.
    - index: Índice del polígono para el que se quiere calcular la superficie.

    Retorna:
    - La superficie del polígono en el índice dado.
    """
    samples = gpd.read_file(samples_path)
    if index in samples.index:
        return samples.loc[index, "geometry"].area
    else:
        return 0

def get_dominant_class(class_dict):
    """
    Identificar la clase dominante y su porcentaje a partir del diccionario de porcentajes.

    Argumentos:
    - class_dict: Diccionario que contiene las clases (códigos numéricos) y sus porcentajes.

    Retorna:
    - El código de la clase dominante y su porcentaje.
    """
    if class_dict:
        dominant_class_code = max(class_dict, key=class_dict.get)
        dominant_class_percentage = class_dict[dominant_class_code]
        return dominant_class_code, dominant_class_percentage
    else:
        return None, 0

def calculate_proportions(class_percentages):
    """
    Calcular la suma de las proporciones de feuillus y de conifères a partir de los datos de una fila.

    Argumentos:
    - class_percentages: Diccionario que contiene los porcentajes de diferentes clases.

    Retorna:
    - La suma de feuillus y conifères.
    """
    feuillus_classes = [11, 12, 13, 14, 15, 16]  # Ejemplo de clases que podrían ser de feuillus
    coniferes_classes = [21, 22, 23, 24, 25, 26, 27]  # Ejemplo de clases que podrían ser de conifères
    
    sum_feuillus = 0
    sum_coniferes = 0
    
    for class_value, percentage in class_percentages.items():
        if class_value in feuillus_classes:
            sum_feuillus += percentage
        elif class_value in coniferes_classes:
            sum_coniferes += percentage
    
    return sum_feuillus, sum_coniferes

def make_decision(surface, sum_feuillus, sum_coniferes, dominant_class_percentage, dominant_class_code):
    """
    Aplicar las reglas de decisión basadas en la superficie y las proporciones.

    Argumentos:
    - surface: La superficie del polígono.
    - sum_feuillus: La suma de las proporciones de feuillus.
    - sum_coniferes: La suma de las proporciones de conifères.
    - dominant_class_percentage: El porcentaje de la clase dominante.
    - dominant_class_code: El código de la clase dominante.

    Retorna:
    - El código predicho para este polígono.
    """
    if surface < 20000:  # Caso superficie < 2 ha
        if sum_feuillus > 75: 
            return 16
        elif sum_coniferes > 75: 
            return 27
        elif sum_coniferes > sum_feuillus: 
            return 28
        else:
            return 29
    else:  # Caso superficie >= 2 ha
        if dominant_class_percentage > 75:
            return int(dominant_class_code)  # Asegurar que sea un número entero
        elif sum_feuillus > 75: 
            return 15
        elif sum_coniferes > 75: 
            return 26
        elif sum_coniferes > sum_feuillus:
            return 28
        else:
            return 29

In [ ]:
import logging

logging.basicConfig(level=logging.INFO, format='%(asctime)s %(message)s')

def apply_decision_rules(df_polygon_classes_percentages, sample_filename):
    """
    Aplicar las reglas de decisión para clasificar cada polígono en función de las proporciones de feuillus y conifères.
    """
    
    polygons = gpd.read_file(sample_filename)
    
    if not df_polygon_classes_percentages.index.equals(polygons.index):
        raise ValueError("Los índices del DataFrame de porcentajes de clases no coinciden con los índices del shapefile.")
    
    predictions = []
    
    for idx, row in df_polygon_classes_percentages.iterrows():
        logging.info(f"Processing polygon index: {idx}")
        class_percentages = row['class_percentages']
        
        sum_feuillus, sum_coniferes = calculate_proportions(class_percentages)
        surface = calculate_surface(sample_filename, idx)
        dominant_class_code, dominant_class_percentage = get_dominant_class(class_percentages)
        
        predicted_class = make_decision(surface, sum_feuillus, sum_coniferes, dominant_class_percentage, dominant_class_code)
        
        predictions.append(predicted_class)
    
    return pd.Series(predictions, index=polygons.index)

In [ ]:
# # Asegúrate de ajustar la ruta del archivo shapefile
sample_filename = "/home/onyxia/work/Depot_Git/results/data/sample/Sample_BD_foret_T31TCJ.shp"  # Reemplázalo con la ruta real
predictions = apply_decision_rules(df_polygon_classes_percentages, sample_filename)

# # Mostrar las predicciones
print(predictions.head())

In [ ]:
print(predictions.unique())

In [ ]:
import os
import geopandas as gpd

# Suponiendo que ya tienes cargado sample_filename y predictions
polygons = gpd.read_file(sample_filename)

# Añadir las predicciones al GeoDataFrame original
polygons["code_predit"] = predictions

# Guardar el archivo con las nuevas predicciones
output_path_samples = os.path.join(my_folder, "classif/carte_essences_echelle_peuplement2.shp")
polygons.to_file(output_path_samples)